In [1]:
with open('data/raw/frankenstein.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[:200])

Letter 1

_To Mrs. Saville, England._


St. Petersburgh, Dec. 11th, 17—.


You will rejoice to hear that no disaster has accompanied the
commencement of an enterprise which you have regarded with such


In [6]:
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ', 'è', 'é', 'ê', 'ô', '—', '‘', '’', '“', '”']
83


In [7]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [10]:
print(decode(encode('hello')))

hello


In [11]:
import torch

In [14]:
data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

tensor([32, 51, 66, 66, 51, 64,  1,  9,  0,  0, 46, 39, 61,  1, 33, 64, 65,  7,
         1, 38, 47, 68, 55, 58, 58, 51,  5,  1, 25, 60, 53, 58, 47, 60, 50,  7,
        46,  0,  0,  0, 38, 66,  7,  1, 36, 51, 66, 51, 64, 65, 48, 67, 64, 53,
        54,  5,  1, 24, 51, 49,  7,  1,  9,  9, 66, 54,  5,  1,  9, 15, 78,  7,
         0,  0,  0, 43, 61, 67,  1, 69, 55, 58, 58,  1, 64, 51, 56, 61, 55, 49,
        51,  1, 66, 61,  1, 54, 51, 47, 64,  1])


In [15]:
n = int(0.8 * len(data))
train_data = data[:n]
test_data = data[n:]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

In [79]:
def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    print(ix)

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    print("x:", x)
    print("y:", y)

    return x, y

In [81]:
x, y = get_batch('train')

tensor([200010,  94489,  73247, 214509])
x: tensor([[50,  1, 66, 54, 51,  1, 65, 51],
        [65, 66, 61, 69, 51, 50,  1, 51],
        [50,  1, 65, 66, 64, 51, 60, 53],
        [ 1, 66, 61,  0, 61, 60, 51,  1]], device='cuda:0')
y: tensor([[ 1, 66, 54, 51,  1, 65, 51, 47],
        [66, 61, 69, 51, 50,  1, 51, 70],
        [ 1, 65, 66, 64, 51, 60, 53, 66],
        [66, 61,  0, 61, 60, 51,  1, 65]], device='cuda:0')


'NVIDIA GeForce RTX 3060 Laptop GPU'

In [18]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, 'target is', target)

when input is tensor([32]) target is tensor(51)
when input is tensor([32, 51]) target is tensor(66)
when input is tensor([32, 51, 66]) target is tensor(66)
when input is tensor([32, 51, 66, 66]) target is tensor(51)
when input is tensor([32, 51, 66, 66, 51]) target is tensor(64)
when input is tensor([32, 51, 66, 66, 51, 64]) target is tensor(1)
when input is tensor([32, 51, 66, 66, 51, 64,  1]) target is tensor(9)
when input is tensor([32, 51, 66, 66, 51, 64,  1,  9]) target is tensor(0)


cuda


In [24]:
randint = torch.randint(-100, 100, (6,))
print(randint)

tensor([-29,  48,  77,  43, -40, -90])


In [25]:
import time

In [42]:
start_time = time.time()
zeros = torch.zeros(1, 1)
end_time = time.time()

elapsed_time = end_time - start_time
print("CPU Time:", elapsed_time)


CPU Time: 0.0006954669952392578


In [43]:
import numpy as np

In [53]:
torch_rand1 = torch.rand(100, 100, 100, 100).to(device=device)
torch_rand2 = torch.rand(100, 100, 100, 100).to(device=device)

np_rand1 = torch.rand(100, 100, 100, 100)
np_rand2 = torch.rand(100, 100, 100, 100)

start_time = time.time()

rand = (torch_rand1 @ torch_rand2)

end_time = time.time()

elapsed_time = end_time - start_time

print("GPU Time:", elapsed_time)

start_time = time.time()

rand = np.multiply(np_rand1, np_rand2)

end_time = time.time()

elapsed_time = end_time - start_time

print("CPU Time:", elapsed_time)

GPU Time: 0.003602743148803711
CPU Time: 0.3079061508178711


In [64]:
vocab_size = len(chars)
embedding_dim = 100
embedding = torch.nn.Embedding(vocab_size, embedding_dim)

input_indices = torch.LongTensor([1, 5, 3, 2])

embedded_output = embedding(input_indices)

print(embedded_output.shape)

torch.Size([4, 100])


In [69]:
a = torch.tensor([[1, 2], [3, 4], [5, 6]])
b = torch.tensor([[7, 8, 9], [10, 11, 12]])
print(a @ b)

tensor([[ 27,  30,  33],
        [ 61,  68,  75],
        [ 95, 106, 117]])


In [72]:
int_64 = torch.randint(100, (10, 100), dtype=torch.float32)
float_32 = torch.rand(100, 100)

result = int_64 @ float_32
print(result.shape)

torch.Size([10, 100])
